## Quick Start Exploration

1. Spleeter can separate audio into 2, 4, or 5 stems
2. Dependent on installation of *ffmpeg* & *libsndfile*
3. If you have issues with ffmpeg probe, pip uninstall ffmpeg-python then pip install ffmpeg-python
4. pip install spleeter
5. There is a conda install (and a conda install for running on GPU) but I was not able to get it to work

In [9]:
from spleeter.separator import Separator
from spleeter.audio.adapter import  get_default_audio_adapter
import librosa
from IPython.display import Audio, display
import numpy as np
import youtube_dl
import os

## 4-stem implementation with one song

In [ ]:
separator = Separator('spleeter:4stems')

In [ ]:
! pwd

In [ ]:
track_dir = '../raw_data/small_sample/train'
track_name = "James May - On The Line.stem.mp4"
audio, rate = librosa.load(f'{track_dir}/{track_name}', mono=False, sr=None)
rate

In [ ]:
audio_loader = get_default_audio_adapter()
waveform, _ = audio_loader.load(f'{track_dir}/{track_name}', sample_rate=rate)

In [ ]:
%%time
prediction = separator.separate(waveform)

In [ ]:
# When ready to do multiple predictions and save output, uncomment line below:

# separator.separate_to_file('/path/to/audio', '/path/to/output/directory')

In [10]:
def display_audio(prediction):
    for key, val in prediction.items():
        print(key)
        display(Audio(val.T, rate=44100))

In [ ]:
display_audio(prediction)

## 5-stem implementation with one song

In [2]:
separator_5 = Separator('spleeter:5stems')

In [3]:
url = "gxEPV4kolz0" #@param {type:"string"}

In [ ]:
def my_hook(d):
    if d['status'] == 'finished':
        print('Done downloading...')


ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
        'preferredquality': '44100',
    }],
    'outtmpl': '%(title)s.wav',
    'progress_hooks': [my_hook],
}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    info = ydl.extract_info(url, download=False)
    status = ydl.download([url])

audio, rate = librosa.load(info.get('title', None) + '.wav', sr=44100, mono=False)
print(audio.shape)
display(Audio(audio, rate=rate))

[youtube] gxEPV4kolz0: Downloading webpage
[youtube] gxEPV4kolz0: Downloading webpage
[download] Billy Joel - Piano Man (Video).wav has already been downloaded
[download] 100% of 5.57MiB
Done downloading...
[ffmpeg] Post-process file Billy Joel - Piano Man (Video).wav exists, skipping


c:\users\njeri\.venvs\mixs_env\lib\site-packages\librosa\core\audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


(2, 15062017)


In [ ]:
audio_loader = get_default_audio_adapter()
waveform_5, _ = audio_loader.load(info.get('title', None) + '.wav', sample_rate=44100)

In [ ]:
%%time
prediction_5 = separator_5.separate(waveform_5)

In [ ]:
display_audio(prediction_5)

## 4-stem implementation with multiple songs

In [ ]:
separator_4 = Separator('spleeter:4stems')

In [ ]:
%%time
track_dir = '../raw_data/small_sample'
audio_loader = get_default_audio_adapter()
predictions = []
for track in os.listdir(track_dir):
    if track.endswith(".mp4"):
        audio, rate = librosa.load(f'{track_dir}/{track}', mono=False, sr=None)
        waveform, _ = audio_loader.load(f'{track_dir}/{track}', sample_rate=rate)
        predictions.append(separator_4.separate(waveform))

len(predictions)

In [ ]:
predictions[17]

## 5-stem implementation with multiple songs

In [11]:
separator_5 = Separator('spleeter:5stems')

In [12]:
%%time
track_dir = '../raw_data/small_sample'
audio_loader = get_default_audio_adapter()
predictions_5 = []
for track in os.listdir(track_dir):
    if track.endswith(".mp4"):
        audio, rate = librosa.load(f'{track_dir}/{track}', mono=False, sr=None)
        waveform, _ = audio_loader.load(f'{track_dir}/{track}', sample_rate=rate)
        predictions_5.append(separator_5.separate(waveform))

len(predictions_5)

c:\users\njeri\.venvs\mixs_env\lib\site-packages\librosa\core\audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


INFO:tensorflow:Apply unet for vocals_spectrogram
INFO:tensorflow:Apply unet for piano_spectrogram
INFO:tensorflow:Apply unet for drums_spectrogram
INFO:tensorflow:Apply unet for bass_spectrogram
INFO:tensorflow:Apply unet for other_spectrogram
INFO:tensorflow:Restoring parameters from pretrained_models\5stems\model
Wall time: 8min 11s


20

In [13]:
predictions_5[14]

{'vocals': array([[-5.9437498e-06,  2.7136530e-06],
        [-1.0363790e-05,  3.4015993e-06],
        [-9.4779962e-06,  1.7994013e-06],
        ...,
        [ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00]], dtype=float32),
 'piano': array([[-7.79070746e-08,  9.77548495e-08],
        [-1.61176175e-07,  1.11829479e-07],
        [-1.13123804e-07,  5.66841578e-08],
        ...,
        [ 0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00]], dtype=float32),
 'drums': array([[-6.1507666e-07,  5.6881100e-07],
        [-7.6188149e-07,  4.4469303e-07],
        [-7.5826102e-07,  3.8342640e-07],
        ...,
        [ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00]], dtype=float32),
 'bass': array([[-8.1884895e-07,  8.6586368e-07],
        [-9.2851548e-07,  8.7294569e-07],
        [-8.9386748e-0